In [195]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [196]:
pd.options.display.max_columns=200
pd.options.display.max_rows=200
pd.options.display.max_info_columns=200

In [197]:
df=pd.read_csv("sf_airbnb listings.csv")
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,958,https://www.airbnb.com/rooms/958,2.019060e+13,6/2/19,"Bright, Modern Garden Unit - 1BR/1B",New update: the house next door is under const...,"Newly remodeled, modern, and bright garden uni...",New update: the house next door is under const...,none,*Quiet cul de sac in friendly neighborhood *St...,Due to the fact that we have children and a do...,*Public Transportation is 1/2 block away. *Ce...,*Full access to patio and backyard (shared wit...,A family of 4 lives upstairs with their dog. N...,* No Pets - even visiting guests for a short t...,NaN,NaN,https://a0.muscache.com/im/pictures/b7c2a199-4...,NaN,1169,https://www.airbnb.com/users/show/1169,Holly,7/31/08,"San Francisco, California, United States",We are a family with 2 boys born in 2009 and 2...,within a few hours,100%,NaN,t,https://a0.muscache.com/im/pictures/efdad96a-3...,https://a0.muscache.com/im/pictures/efdad96a-3...,Duboce Triangle,1,1,"[''''email'''', ''''phone'''', ''''facebook'''...",t,t,"San Francisco, CA, United States",Lower Haight,Western Addition,NaN,San Francisco,CA,94117,San Francisco,"San Francisco, CA",US,United States,37.76931,-122.43386,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,Cable TV,Internet,Wifi,Kitchen,Pets live o...",NaN,$170.00,"$1,120.00","$4,200.00",$100.00,$100.00,2,$25.00,1,30,1,1,30,30,1.0,30.0,2 months ago,t,1,3,7,92,6/2/19,192,56,7/23/09,5/16/19,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,STR-0001256,{SAN FRANCISCO},t,f,moderate,f,f,1,1,0,0,1.60
1,5858,https://www.airbnb.com/rooms/5858,2.019060e+13,6/2/19,Creative Sanctuary,NaN,We live in a large Victorian house on a quiet ...,We live in a large Victorian house on a quiet ...,none,I love how our neighborhood feels quiet but is...,All the furniture in the house was handmade so...,The train is two blocks away and you can stop ...,"Our deck, garden, gourmet kitchen and extensiv...",NaN,"Please respect the house, the art work, the fu...",NaN,NaN,https://a0.muscache.com/im/pictures/17714/3a7a...,NaN,8904,https://www.airbnb.com/users/show/8904,Philip And Tania,3/2/09,"San Francisco, California, United States",Philip: English transplant to the Bay Area and...,within a day,100%,NaN,f,https://a0.muscache.com/im/users/8904/profile_...,https://a0.muscache.com/im/users/8904/profile_..

In [198]:
df=df.drop(["id","listing_url","scrape_id","last_scraped","name","summary","space","description","experiences_offered","neighborhood_overview","notes","transit","access","interaction",
            "house_rules","thumbnail_url","medium_url","picture_url","xl_picture_url","host_id","host_url","host_name","host_since","host_location","host_about","host_response_time","host_response_rate",
            "host_acceptance_rate","host_is_superhost","host_thumbnail_url","host_picture_url","host_neighbourhood","host_verifications","street","neighbourhood","neighbourhood_cleansed","neighbourhood_group_cleansed",
            "city","state","zipcode","market","smart_location","country_code","country","amenities","square_feet","calendar_updated","calendar_last_scraped","first_review","last_review","license",
            "jurisdiction_names","cancellation_policy","weekly_price","monthly_price"],axis=1)

In [199]:
df

,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,1,1,t,t,37.76931,-122.43386,t,Apartment,Entire home/apt,3,1.0,1.0,2.0,Real Bed,$170.00,$100.00,$100.00,2,$25.00,1,30,1,1,30,30,1.0,30.0,t,1,3,7,92,192,56,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,t,f,f,f,1,1,0,0,1.60
1,2,2,t,t,37.74511,-122.42102,t,Apartment,Entire home/apt,5,1.0,2.0,3.0,Real Bed,$235.00,NaN,$100.00,2,$0.00,30,60,30,30,60,60,30.0,60.0,t,0,0,0,0,111,0,98.0,10.0,10.0,10.0,10.0,10.0,9.0,t,f,f,f,f,1,1,0,0,0.90
2,10,10,t,t,37.76669,-122.45250,t,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,$200.00,$50.00,1,$12.00,32,60,32,32,60,60,32.0,60.0,t,30,60,90,365,17,0,85.0,8.0,8.0,9.0,9.0,9.0,8.0,t,f,f,f,f,9,0,9,0,0.14
3,10,10,t,t,37.76487,-122.45183,t,Apartment,Private room,2,4.0,1.0,1.0,Real Bed,$65.00,$200.00,$50.00,1,$12.00,32,90,32,32,90,90,32.0,90.0,t,11,41,71,346,8,1,93.0,9.0,9.0,10.0,10.0,9.0,9.0,t,f,f,f,f,9,0,9,0,0.14
4,2,2,t,t,37.77525,-122.43637,t,House,Entire home/apt,5,1.5,2.0,2.0,Real Bed,$685.00,$0.00,$225.00,2,$150.00,4,1125,4,4,1125,1125,4.0,1125.0,t,30,60,90,90,27,1,97.0,10.0,10.0,10.0,10.0,10.0,9.0,t,f,f,t,t,2,2,0,0,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7570,2,2,t,f,37.75227,-122.42785,f,Condominium,Entire home/apt,2,1.0,2.0,2.0,Real Bed,$475.00,NaN,NaN,1,$0.00,3,1125,3,4,1125,1125,3.2,1125.0,t,26,56,86,86,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,f,f,f,1,1,0,0,NaN
7571,92,92,t,f,37.75455,-122.42382,t,Apartment,Entire home/apt,3,1.0,1.0,1.0,Real Bed,$115.00,$0.00,$95.00,1,$0.00,30,150,30,30,150,150,30.0,150.0,t,23,53,83,173,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,f,f,f,f,92,91,1,0,NaN
7572,2,2,t,t,37.75803,-122.42080,f,House,Entire home/apt,6,1.5,2.0,2.0,Real Bed,$500.00,$0.00,$75.00,4,$25.00,1,10,1,1,10,10,1.0,10.0,t,7,9,9,9,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,f,f,f,2,1,1,0,NaN
7573,3,3,t,f,37.76089,-122.50258,t,House,Entire home/apt,6,1.0,2.0,3.0,Real Bed,$180.00,"$1,000.00",$150.00,4,$5.00,30,1125,30,30,1125,1125,30.0,1125.0,t,16,46,76,107,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t,t,f,f,f,3,2,1,0,NaN


In [200]:
df.security_deposit.value_counts()

$0.00         1820
$500.00       1262
$200.00        410
$1,000.00      381
$100.00        356
$300.00        350
$250.00        276
$150.00        158
$400.00        153
$2,000.00      121
$2,500.00       85
$1,500.00       83
$350.00         72
$5,000.00       61
$600.00         57
$800.00         47
$3,000.00       36
$750.00         36
$700.00         29
$450.00         21
$3,500.00       21
$650.00         12
$900.00         11
$4,000.00        9
$1,200.00        8
$125.00          8
$95.00           8
$199.00          7
$120.00          7
$1,250.00        7
$950.00          6
$175.00          6
$180.00          5
$375.00          5
$550.00          5
$499.00          4
$299.00          4
$1,800.00        4
$1,700.00        3
$4,500.00        3
$130.00          2
$1,600.00        2
$2,600.00        2
$5,100.00        2
$3,300.00        2
$325.00          2
$1,300.00        2
$1,750.00        2
$2,700.00        2
$225.00          2
$1,900.00        2
$680.00          2
$1,400.00   

In [201]:
df.isnull().sum()

host_listings_count                                0
host_total_listings_count                          0
host_has_profile_pic                               0
host_identity_verified                             0
latitude                                           0
longitude                                          0
is_location_exact                                  0
property_type                                      0
room_type                                          0
accommodates                                       0
bathrooms                                         18
bedrooms                                           2
beds                                               7
bed_type                                           0
price                                              0
security_deposit                                1543
cleaning_fee                                     835
guests_included                                    0
extra_people                                  

In [202]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7575 entries, 0 to 7574
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   host_listings_count                           7575 non-null   int64  
 1   host_total_listings_count                     7575 non-null   int64  
 2   host_has_profile_pic                          7575 non-null   object 
 3   host_identity_verified                        7575 non-null   object 
 4   latitude                                      7575 non-null   float64
 5   longitude                                     7575 non-null   float64
 6   is_location_exact                             7575 non-null   object 
 7   property_type                                 7575 non-null   object 
 8   room_type                                     7575 non-null   object 
 9   accommodates                                  7575 non-null   i

In [203]:
df.security_deposit.value_counts()

$0.00         1820
$500.00       1262
$200.00        410
$1,000.00      381
$100.00        356
$300.00        350
$250.00        276
$150.00        158
$400.00        153
$2,000.00      121
$2,500.00       85
$1,500.00       83
$350.00         72
$5,000.00       61
$600.00         57
$800.00         47
$3,000.00       36
$750.00         36
$700.00         29
$450.00         21
$3,500.00       21
$650.00         12
$900.00         11
$4,000.00        9
$1,200.00        8
$125.00          8
$95.00           8
$199.00          7
$120.00          7
$1,250.00        7
$950.00          6
$175.00          6
$180.00          5
$375.00          5
$550.00          5
$499.00          4
$299.00          4
$1,800.00        4
$1,700.00        3
$4,500.00        3
$130.00          2
$1,600.00        2
$2,600.00        2
$5,100.00        2
$3,300.00        2
$325.00          2
$1,300.00        2
$1,750.00        2
$2,700.00        2
$225.00          2
$1,900.00        2
$680.00          2
$1,400.00   

In [233]:
df["security_deposit"]=df["security_deposit"].fillna("0.00")
df["security_deposit"]=df["security_deposit"].to_string()


df["security_deposit"]=df["security_deposit"].str.strip("$").str.replace(",",".")



print(df["security_deposit"])






#df.security_deposit.value_counts()

0       0       0       0       0       0         100....
1       0       0       0       0       0         100....
2       0       0       0       0       0         100....
3       0       0       0       0       0         100....
4       0       0       0       0       0         100....
                              ...                        
7570    0       0       0       0       0         100....
7571    0       0       0       0       0         100....
7572    0       0       0       0       0         100....
7573    0       0       0       0       0         100....
7574    0       0       0       0       0         100....
Name: security_deposit, Length: 7575, dtype: object


In [205]:
df.cleaning_fee.value_counts()

$100.00     654
$150.00     476
$50.00      416
$75.00      304
$0.00       288
$95.00      264
$200.00     263
$25.00      261
$125.00     253
$40.00      225
$60.00      204
$30.00      204
$250.00     193
$80.00      191
$120.00     144
$35.00      143
$240.00     122
$45.00      118
$20.00      106
$90.00      101
$70.00       92
$300.00      88
$65.00       85
$85.00       82
$99.00       80
$175.00      78
$15.00       58
$10.00       56
$101.00      54
$148.00      54
$160.00      52
$330.00      50
$55.00       46
$140.00      46
$29.00       45
$225.00      40
$110.00      38
$180.00      35
$280.00      33
$39.00       32
$244.00      27
$69.00       25
$130.00      24
$350.00      24
$79.00       22
$89.00       21
$185.00      20
$370.00      20
$500.00      20
$155.00      17
$49.00       15
$195.00      14
$135.00      14
$76.00       13
$59.00       13
$275.00      13
$115.00      12
$400.00      12
$145.00      12
$165.00      11
$149.00      10
$105.00       9
$88.00  

In [206]:
df["cleaning_fee"]=df["cleaning_fee"].str.strip("$").str.replace(",",".").astype(float)



df.cleaning_fee.value_counts()

100.0    654
150.0    476
50.0     416
75.0     304
0.0      288
95.0     264
200.0    263
25.0     261
125.0    253
40.0     225
60.0     204
30.0     204
250.0    193
80.0     191
120.0    144
35.0     143
240.0    122
45.0     118
20.0     106
90.0     101
70.0      92
300.0     88
65.0      85
85.0      82
99.0      80
175.0     78
15.0      58
10.0      56
101.0     54
148.0     54
160.0     52
330.0     50
55.0      46
140.0     46
29.0      45
225.0     40
110.0     38
180.0     35
280.0     33
39.0      32
244.0     27
69.0      25
130.0     24
350.0     24
79.0      22
89.0      21
185.0     20
370.0     20
500.0     20
155.0     17
49.0      15
195.0     14
135.0     14
76.0      13
59.0      13
275.0     13
115.0     12
400.0     12
145.0     12
165.0     11
149.0     10
105.0      9
88.0       9
190.0      8
189.0      8
68.0       8
5.0        8
205.0      8
19.0       7
320.0      7
199.0      7
208.0      7
170.0      6
58.0       6
220.0      6
28.0       6
33.0       5

In [227]:
df["bathrooms"]=df.bathrooms.mean()
df["bedrooms"]=df.bedrooms.mean()
df["beds"]=df.beds.mean()
#df["security_deposit"]=df.security_deposit.mean()
df["cleaning_fee"]=df.cleaning_fee.mean()
df["review_scores_rating"]=df.review_scores_rating.mean()
df["review_scores_accuracy"]=df.review_scores_accuracy.mean()
df["review_scores_cleanliness"]=df.review_scores_cleanliness.mean()
df["review_scores_checkin"]=df.review_scores_checkin.mean()
df["review_scores_communication"]=df.review_scores_communication.mean()
df["review_scores_location"]=df.review_scores_location.mean()
df["review_scores_value"]=df.review_scores_value.mean()
df["reviews_per_month"]=df.reviews_per_month.mean()